The code below simulates the Cramer-Lundberg Model using jump times.

In [ ]:
# define parameters as given in the project description
x0 = 10          # initial surplus
LAMBDA = 2       # claim arrival rate
k = 2            # shape parameter of Gamma distribution
theta = 1            # scale parameter of Gamma distribution
ETA = 0.1        # safety loading factor
T = 5            # time horizon

# calculate expected claim size and premium rate
EY = k * theta  # mean of gamma(A, B) distribution
c = (1 + ETA) * LAMBDA * EY  # premium rate

import numpy as np
import matplotlib.pyplot as plt

def simulate_cl_jump_times(x0, c, lambda_, k, theta, T):
    """
    This function uses the Cramer-Lundberg model to simulate the wealth process of an insurance company over a fixed time interval [0,T).

    Assumptions:
    - The number of claims on [0,T) is distributed as a Poisson(lambda_*T) random variable.
    - The times at which claims occur are distributed as Uniform(0,T) random variables.
    - Claim sizes are distributed as Gamma(k, theta) random variables.
    - The wealth of the company only changes at times when a new claim is made, and remains stationary otherwise.

    Input parameters:
    - x0 (float): initial wealth.
    - c (float): premium rate.
    - lambda_ (float): claim arrival rate.
    - k (float): shape parameter of the gamma distribution for claim sizes.
    - theta (float): scale parameter of the gamma distribution for claim sizes.
    - T (float): length of the fixed time interval [0,T].

    Output parameters:
    - jump_times (np.ndarray): sorted array of jump times within [0, T].
    - surplus_at_jumps (np.ndarray): array of surplus values at each jump time.
    """
    # simulate N_T, the number of claims over [0, T), as a realization of a poisson(lambda_*T) distributed rv
    N_T = np.random.poisson(lam=lambda_ * T)

    # generate N_T jump-times over the [0,T) time interval as realizations from a uniform[0,T) distribution, and sort the jump times in order of occurrence
    jump_times = []
    if N_T > 0:
        for i in range(N_T):
            jump_times.append(np.random.uniform(low=0,high=T))
        jump_times = sorted(jump_times)
    else: #if N_T = 0 then the array of jumptimes is empty
        jump_times = np.array([])

    # initialize the surplus array
    surplus_at_jumps = []

    # initialize the initial surplus
    initial_surplus = x0

    # initialize cumulative sum of claim sizes
    cumsum_claim_sizes = 0

    # simulate the cramer-lundberg model by updating the surplus at each jump-time
    for jump_time in jump_times:

        # compute the premium income at the current jump time
        premium_income = c * jump_time

        # simulate claim size at the current jump time, and update the cumulative sum of claim sizes up to the current jump time
        Y_i = np.random.gamma(shape=k, scale=theta)
        cumsum_claim_sizes += Y_i

        # compute the surplus at the current jump-time by applying Cramer-Lundberg model
        current_surplus = initial_surplus + premium_income - cumsum_claim_sizes
        # record the surplus after the claim
        surplus_at_jumps.append(current_surplus)

    return jump_times, np.array(surplus_at_jumps)

num_simulations = 25

# array to store surplus values over time for all simulations
time_grid = np.linspace(0, T, 1000)
surplus_values = np.zeros((len(time_grid), num_simulations))

# perform and plot 25 simulations
for sim in range(num_simulations):
    jump_times, surplus_at_jumps = simulate_cl_jump_times(x0, c, LAMBDA, k, theta, T)

    # initialize lists for plotting
    plot_times = [0]
    plot_surplus = [x0]

    # add jump times and surplus after each jump
    for t, s in zip(jump_times, surplus_at_jumps):
        plot_times.append(t)
        plot_surplus.append(s)

    # calculate surplus at time T
    if len(jump_times) > 0:
        last_surplus = surplus_at_jumps[-1]
    else:
        last_surplus = x0

    plot_times.append(T)
    plot_surplus.append(last_surplus)

    # interpolate surplus values onto time grid
    surplus_values[:, sim] = np.interp(time_grid, plot_times, plot_surplus)

# plotting all simulations as step-wise processes
plt.figure(figsize=(12, 6))
for sim in range(num_simulations):
    plt.step(time_grid, surplus_values[:, sim], where='post', alpha=0.2)

# calculate and plot median, 10th and 90th percentiles across all simulations
median_surplus = np.median(surplus_values, axis=1)
perc_10 = np.percentile(surplus_values, 10, axis=1)
perc_90 = np.percentile(surplus_values, 90, axis=1)

# plot median and percentiles in black
plt.plot (time_grid, median_surplus, color='black', linestyle='-', label='Median')
plt.plot (time_grid, perc_10, color='black', linestyle='--', label='10th Percentile')
plt.plot (time_grid, perc_90, color='black', linestyle='--', label='90th Percentile')

# plot aesthetics for all simulations
plt.xlabel("Time $t$")
plt.ylabel("Wealth $X_t$")
plt.title(f"Cramer-Lundberg Model Wealth Process (Jump Times Only) Over {num_simulations} Simulations")
plt.legend()
plt.grid(True)
plt.xlim(0, T)

plt.show()

The code below simulates the Cramer-Lundberg Model using a fixed time grid.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def CL_gamma_time_grid(x, c, sigma, lmbda, k, theta, max_time, Ndt, Nsims):
    """
    This function simulates the Cramer-Lundberg Model with Gamma-distributed jumps using a fixed time grid.

    Input parameters:
    - x (float): initial wealth.
    - c (float): premium rate.
    - sigma (float): volatility of the process.
    - lmbda (float): claim arrival rate.
    - k (float): shape parameter of the gamma distribution for claim sizes.
    - theta (float): scale parameter of the gamma distribution for claim sizes.
    - max_time (float): length of the fixed time interval [0, max_time]
    - Ndt (int): number of smaller time intervals within [0, max_time]
    - Nsims (int): number of simulations to use.

    Output parameters:
    - time (ndarray): time points representing the discretized time grid over [0, max_time]
    - X (ndarray): wealth values of simulations over [0, max_time]
    """
    # time step size
    dt = max_time / Ndt

    # array of time points from 0 to max_time
    time = np.linspace(0, max_time, Ndt + 1)

    # initialize wealth array at initial value
    X = np.zeros((Ndt + 1, Nsims))
    X[0, :] = x

    # simulate wealth process over each time step
    for i in range(Ndt):

        # generate diffusion component
        G = np.random.normal(0, sigma*np.sqrt(dt), size = Nsims)

        # generate random values to determine whether a jump occurs
        U = np.random.uniform(size = Nsims)
        jump = U < (1-np.exp(-lmbda*dt)) # boolean array indicating jumps

        # generate jump sizes from a Gamma(k,theta) distribution
        jump_size = np.random.gamma(k, theta, size = Nsims)

        # Update wealth for next time step
        # increment wealth by deterministic drift c*dt, diffusion G, and jump size (if jump occurs)
        X[i + 1,:] = X[i, :] + c*dt + G - jump*jump_size

    return time, X


# set parameters (see section 3 of project report) and run simulation
time, X = CL_gamma_time_grid(10, 4.4, 1, 2, 2, 1, 5, 1000, 100)

# set up figure for plotting
plt.figure(figsize=(10, 6))

def plotsims(time, X, Nsims):
    """
    This function plots simulations including median, 10th and 90th quantiles, along with sample paths.

    Input parameters:
    - time (ndarray): array of time points.
    - X (ndarray): array of wealth values at each time point for each sample path.
    - Nsims (int): number of simulations paths to plot.
    """
    # calculate the median, 10th and 90th quantiles for wealth at each time step
    median = np.median(X, axis=1)
    lower_quantile = np.percentile(X, 10, axis=1)
    upper_quantile = np.percentile(X, 90, axis=1)

    # plot median and quantiles in black
    plt.plot(time, median, color='black', linestyle='-', label='Median')
    plt.plot(time, lower_quantile, color='black', linestyle='--', label='10th Percentile')
    plt.plot(time, upper_quantile, color='black', linestyle='--', label='90th Percentile')

    # plot a few sample paths in color
    for i in range(min(5, Nsims)):  # plot a few paths
        plt.plot(time, X[:, i], alpha=0.7)

    # add labels, title, legend and grid to plot figure
    plt.xlabel("Time")
    plt.ylabel("Wealth")
    plt.title("Cramer-Lundberg Model Simulation using Fixed Time Grid")
    plt.legend()
    plt.grid()
    plt.show()

# plot simulations on figure
plotsims(time, X, 100)

The code below explores how adjusting parameters affects the wealth process.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

def CL_gamma_time_grid(x, c, sigma, lmbda, k, theta, max_time, Ndt, Nsims):
    """
    Simulate the Cramer-Lundberg Model with Gamma-distributed jumps using a fixed time grid.

    Input parameters:
    - x (float): initial wealth.
    - c (float): premium rate.
    - sigma (float): volatility of the process.
    - lmbda (float): claim arrival rate.
    - k (float): shape parameter of the gamma distribution for claim sizes.
    - theta (float): scale parameter of the gamma distribution for claim sizes.
    - max_time (float): length of the fixed time interval [0, max_time]
    - Ndt (int): number of smaller time intervals within [0, max_time]
    - Nsims (int): number of simulations to use.

    Output parameters:
    - time (ndarray): time points representing the discretized time grid over [0, max_time]
    - X (ndarray): wealth values of simulations over [0, max_time]
    """

    # time step size
    dt = max_time / Ndt

    # array of time points from 0 to max_time
    time = np.linspace(0, max_time, Ndt + 1)

    # initialize wealth array at initial value
    X = np.zeros((Ndt + 1, Nsims))
    X[0, :] = x

    # simulate wealth process over each time step
    for idx in range(Ndt):

        # generate diffusion component
        G = np.random.normal(0, sigma * np.sqrt(dt), size=Nsims)

        # generate random values to determine whether a jump occurs
        U = np.random.uniform(size=Nsims)
        jump = U < (1 - np.exp(-lmbda * dt))

        # generate jump sizes from a Gamma(k,theta) distribution
        jump_size = np.random.gamma(k, theta, size=Nsims)

        # Update wealth for next time step
        # increment wealth by deterministic drift c*dt, diffusion G, and jump size (if jump occurs)
        X[idx + 1, :] = X[idx, :] + c * dt + G - jump * jump_size

    return time, X

# set parameters (see section 3 of project report)
x = 10
c = 4.4
sigma = 1
lmbda = 2
k = 2
theta = 1
max_time = 5
Ndt = 1000
Nsims = 50

# create figure and two plots: one for plotting wealth process and one for plotting histogram
fig, (plts_plot, plts_hist) = plt.subplots(2, 1, figsize=(12, 10), gridspec_kw={'height_ratios': [4, 1.5]})
plt.subplots_adjust(left=0.25, bottom=0.30, hspace=0.5)

# run initial simulation
time, X = CL_gamma_time_grid(x, c, sigma, lmbda, k, theta, max_time, Ndt, Nsims)

# plot initial wealth paths
for i in range(Nsims):
    plts_plot.plot(time, X[:, i], alpha=0.4)
plts_plot.set_xlabel("Time")
plts_plot.set_ylabel("Wealth")
plts_plot.set_title("Adjusting Parameters")
plts_plot.grid()

# plot histogram of final wealth values at max_time=5 (T=5)
plts_hist.hist(X[-1, :], bins=30, alpha=0.7, edgecolor='black')
plts_hist.set_xlabel("Wealth at T=5")
plts_hist.set_ylabel("Frequency")

# set up slider locations and properties
slider_height = 0.03
slider_spacing = 0.005

plts_c = plt.axes([0.25, 0.20 - 0 * (slider_height + slider_spacing), 0.65, slider_height])
plts_sigma = plt.axes([0.25, 0.20 - 1 * (slider_height + slider_spacing), 0.65, slider_height])
plts_lmbda = plt.axes([0.25, 0.20 - 2 * (slider_height + slider_spacing), 0.65, slider_height])
plts_k = plt.axes([0.25, 0.20 - 3 * (slider_height + slider_spacing), 0.65, slider_height])
plts_theta = plt.axes([0.25, 0.20 - 4 * (slider_height + slider_spacing), 0.65, slider_height])
plts_Ndt = plt.axes([0.25, 0.20 - 5 * (slider_height + slider_spacing), 0.65, slider_height])

# create sliders for adjusting parameters
slider_c = Slider(plts_c, "c", 0.1, 10.0, valinit=c)
slider_sigma = Slider(plts_sigma, "Sigma", 0.1, 10.0, valinit=sigma)
slider_lmbda = Slider(plts_lmbda, "Lambda", 0.1, 20.0, valinit=lmbda)
slider_k = Slider(plts_k, "Shape", 0.1, 5.0, valinit=k)
slider_theta = Slider(plts_theta, "Scale", 0.1, 2.0, valinit=theta)
slider_Ndt = Slider(plts_Ndt, "Time Step", 100, 2000, valinit=Ndt, valstep=100)

# function to update plots when sliders are adjusted
def adjust(val):
    # get updated values from sliders
    c = slider_c.val
    sigma = slider_sigma.val
    lmbda = slider_lmbda.val
    k = slider_k.val
    theta = slider_theta.val
    Ndt = int(slider_Ndt.val)

    # re-run the simulation with updated parameters
    time, X = CL_gamma_time_grid(x, c, sigma, lmbda, k, theta, max_time, Ndt, Nsims)

    # clear previous plots and re=plot with updated values
    plts_plot.clear()

    for i in range(Nsims):
        plts_plot.plot(time, X[:, i], alpha=0.4)

    # set labels and grid for updated plot
    plts_plot.set_xlabel("Time")
    plts_plot.set_ylabel("Wealth")
    plts_plot.set_title("Adjusting Parameters")
    plts_plot.grid()

    # update histogram of wealth at final time step
    plts_hist.clear()
    plts_hist.hist(X[-1, :], bins=30, alpha=0.7, edgecolor='black')
    plts_hist.set_xlabel("Wealth at T=5")
    plts_hist.set_ylabel("Frequency")

    # re-draw canvas to show updated plots
    fig.canvas.draw_idle()

# link sliders to adjust function
slider_c.on_changed(adjust)
slider_sigma.on_changed(adjust)
slider_lmbda.on_changed(adjust)
slider_k.on_changed(adjust)
slider_theta.on_changed(adjust)
slider_Ndt.on_changed(adjust)

# display plot with sliders
plt.show()


This code simulates the Cramer-Lundberg Model using the SDE approximation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def CL_approx_SDE(x, c, lmbda, EY, EY2, max_time, Ndt, Nsims):
    """
    This function simulates the Cramer-Lundberg Model using an SDE approximation.

    Input parameters:
    - x (float): initial wealth.
    - c (float): premium rate.
    - lmbda (float): claim arrival rate.
    - EY (float): expected value of the claim size.
    - EY2 (float): second moment of the claim size.
    - theta (float): scale parameter of the gamma distribution for claim sizes.
    - max_time (float): length of the fixed time interval [0, max_time]
    - Ndt (int): number of smaller time intervals within [0, max_time]
    - Nsims (int): number of simulations to use.

    Output parameters:
    - time (ndarray): time points representing the discretized time grid over [0, max_time]
    - X (ndarray): wealth values of simulations over [0, max_time]
    """

    # time step size
    dt = max_time / Ndt

    # array of time points from 0 to max_time
    time = np.linspace(0, max_time, Ndt + 1)

    # initialize wealth array
    X = np.zeros((Ndt + 1, Nsims))
    X[0, :] = x

    # simulate wealth process over each time step
    for i in range(Ndt):
        t = time[i]

        # generate standard normal random numbers for Brownian Motion component
        W = np.random.randn(Nsims)

        # calculate drift component
        drift = (c - lmbda * EY) * dt

        # calculate diffusion component
        diffusion = -np.sqrt(lmbda * EY2) * np. sqrt(dt) * W

        # update wealth for next time step
        X[i + 1, :] = X[i, :] + drift + diffusion

    return time, X

# set parameters for simulation
x = 10
c = 4.4
lmbda = 2
EY = 2
EY2 = 6
max_time = 5
Ndt = 1000
Nsims = 100

# run sde approximation simulation
time_sde, X_sde = CL_approx_SDE(x, c, lmbda, EY, EY2, max_time, Ndt, Nsims)

# set up figure for plotting
plt.figure(figsize=(10, 6))


def plotsims_sde(time, X, Nsims):
    """
    This function plots simulations including median, 10th and 90th quantiles, along with sample paths.

    Input parameters:
    - time (ndarray): array of time points.
    - X (ndarray): array of wealth values at each time point for each sample path.
    - Nsims (int): number of simulations paths to plot.
    """

    # calculate the median, 10th and 90th quantiles for wealth at each time step
    median = np.median(X, axis=1)
    lower_quantile = np.percentile(X, 10, axis=1)
    upper_quantile = np.percentile(X, 90, axis=1)

    # plot median and quantiles in black
    plt.plot(time, median, color='black', linestyle='-', label='Median')
    plt.plot(time, lower_quantile, color='black', linestyle='--', label='10th Percentile')
    plt.plot(time, upper_quantile, color='black', linestyle='--', label='90th Percentile')

    # plot a few sample paths in color
    for i in range(min(5, Nsims)):  # plot a few paths
        plt.plot(time, X[:, i], alpha=0.7)

    # add labels, title, legend and grid to plot figure
    plt.xlabel("Time")
    plt.ylabel("Wealth")
    plt.title("Cramer-Lundberg SDE Approximation")
    plt.legend()
    plt.grid()
    plt.show()

# plot simulations on figure
plotsims_sde(time_sde, X_sde, 100)

This code visually compares the results of simulating the CL model over a fixed time grid, and the results of simulating the CL model using the SDE approximation on the time interval [0,5].

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# initial settings
x = 10  #initial wealth
T = 5
n_steps = 1000  # number of steps

# function to calculate derived parameters
def derive_params(lambda_val, k_shape, theta, eta):
    E_Y = k_shape * theta  # mean of Gamma distribution
    E_Y2 = k_shape * (theta**2 + theta**2)  # second moment
    c = (1 + eta) * lambda_val * E_Y  # premium rate
    dt = T / n_steps  # size of each time step
    return c, E_Y, E_Y2, dt

# simulate diffusion approximation for wealth process
def simulate_diffusion(lambda_val, E_Y, E_Y2, c, dt):
    time_grid = np.linspace(0, T, n_steps)
    Xt = np.zeros(n_steps)
    Xt[0] = x
    W_t = np.random.normal(0, np.sqrt(dt), n_steps - 1)
    for i in range(1, n_steps):
        Xt[i] = Xt[i - 1] + (c -lambda_val * E_Y) * dt + np.sqrt(lambda_val * E_Y2) * W_t[i - 1]
    return time_grid, Xt

# simulate grid-based wealth process
def simulate_grid_based(lambda_val, k_shape, theta, c, dt):
    time_grid = np.linspace(0, T, n_steps)
    #claims follow poisson distribution
    Nt = np.random.poisson(lambda_val * dt, n_steps)  #claims per step
    #claim sizes follow Gamma distribution
    claim_sizes = np.random.gamma(k_shape, theta, sum(Nt)) #claim sizes
    Xt = [x]
    claims_idx = 0
    for n in Nt:
        claims_sum = sum(claim_sizes[claims_idx:claims_idx + n])
        claims_idx += n
        Xt.append(Xt[-1] + c * dt - claims_sum)
    return time_grid, Xt[:-1]

# function to compare wealth processes for given parameters
def compare_wealth_processes(lambda_val, k_shape, theta, eta):
    c, E_Y, E_Y2, dt = derive_params(lambda_val, k_shape, theta, eta)

    # simulate both models
    time_grid_diffusion, Xt_diffusion = simulate_diffusion(lambda_val, E_Y, E_Y2, c, dt)
    time_grid_grid, Xt_grid = simulate_grid_based(lambda_val, k_shape, theta, c, dt)

    # plot results for both models in distinct colours
    plt.figure(figsize=(12, 6))
    plt.plot(time_grid_diffusion, Xt_diffusion, label="Diffusion Approximation", color="blue")
    plt.step(time_grid_grid, Xt_grid, label="Grid-Based Simulation", color="orange", linestyle="--", where="post")
    plt.title(f"Wealth Process Comparison: Simulation vs. Approximation\nλ={lambda_val}, k={k_shape}, θ={theta}, η={eta}")
    plt.xlabel("Time")
    plt.ylabel("Wealth Process ($X_t$)")
    plt.legend()
    plt.grid()
    plt.show()

#plot examples for changing parameters
parameter_sets = [
    {"lambda_val": 2, "k_shape": 2, "theta": 1, "eta": 0.9},
    {"lambda_val": 5, "k_shape": 2, "theta": 1, "eta": 0.9},
    {"lambda_val": 2, "k_shape": 3, "theta": 1, "eta": 0.9},
    {"lambda_val": 5, "k_shape": 2, "theta": 1, "eta": 0.9},
]

# generating plot for each parameter set
for params in parameter_sets:
    compare_wealth_processes(**params)

This code computes an Importance Sampling Estimate of the Ultimate ruin probability of the CL Model, given the parameters defined in Section 6.6.1 of our report.

In [ ]:
import numpy as np
from scipy.optimize import root_scalar
from scipy.stats import norm
def CL_ultimate_ruin_probability_importance_sampling_estimator(X0, c, alpha, lambda_, Nsims, R_star):
    '''This function computes an importance sampling estimate of the ultimate ruin probability of the CL model, given the parameters defined in Section 6.6.1 of our report.
    It also outputs several other variables, which are described below.
    Input variables:
    - X0: Initial Surplus
    - c: Premium
    - alpha: Expected Claim Size
    - lambda_: Intensity of the Poisson RV Nt
    - Nsims: Number of Desired Simulations
    - R_star: The optimal constant which ensures the Likelihood ratio (described in Section [] of the Report) is well-defined
    Output variables:
    - ruin_prob: The ruin probability importance sampling estimate
    - ci_lower: The lower bound of the 95% confidence interval for ruin_prob
    - ci_upper: The upper bound of the 95% confidence interval for ruin_prob
    '''
    count = 0
    # estimate ruin probability
    estimates = np.zeros(Nsims)
    for i in range(Nsims):
        S_i = 0 #initialize S_i, the total change in wealth between the point in time where the ith claim occurs, and the wealth at t = 0 (which is X0)
        while S_i > -X0: #this loop simulates claim arrivals until S_i is less than the negative of the initial surplus; in other words, it simulates claim arrivals until ruin has occurred
            delta_t_i = np.random.exponential(scale = 1/lambda_)
            Y = np.random.exponential(scale = 1/alpha)
            S_i += - c*delta_t_i + Y
        count += 1
        estimates[i] = np.exp(R_star * S_i) #append this realization of e^(R_star(S_i)) to the estimate array

    # compute mean, standard error, and 95% confidence intervals of the estimate
    ruin_prob = np.mean(estimates) #compute the sample mean of all Nsims realizations of e^(R_star(S_i))
    ruin_prob_se = (np.std(estimates, ddof=1) / np.sqrt(Nsims))
    ci_lower = ruin_prob - ruin_prob_se* norm.ppf(0.975)
    ci_upper = ruin_prob + ruin_prob_se* norm.ppf(0.975)
    ci= [ci_lower, ci_upper]

    return ruin_prob, ruin_prob_se, ci
analytical_solution = (0.5/(1*1))*np.exp(-(1-(0.5/1))*20)
print(f"Analytical Solution: {analytical_solution}")
ultimate_ruin_prob, ultimate_ruin_prob_se, ci = CL_ultimate_ruin_probability_importance_sampling_estimator(20, 1,1,0.5, 10000,0.5)
print(f"Importance sampling estimate: {ultimate_ruin_prob}")
print(f"Absolute Difference between Analytical solution and Importance sampling estimate: {np.abs(analytical_solution - ultimate_ruin_prob)}")
print(f"Standard Error of Importance Sampling estimate: {ultimate_ruin_prob_se}")
print(f"95% Confidence Interval for Importance sampling estimate: {ci}")